In [1]:
import pandas as pd 
import numpy as np 
import contractions 
import nltk 
from nltk import word_tokenize
from bs4 import BeautifulSoup

In [2]:
def preprocess(text):
    soup = BeautifulSoup(text,'html.parser')
    text = soup.get_text()
    words = word_tokenize(text)
    text = " ".join([word.lower() for word in words if word not in stop])
    text = re.sub(r'[^a-zA-Z0-9\s]',"",text,re.I)
    text = text.strip()
    text = contractions.fix(text)
    return text

In [3]:
dataset = pd.read_excel(r"Electronics_data.xlsx")

In [4]:
dataset.head()

,Unnamed: 0,sentiment,title,Reviews
0,0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...


In [5]:
dataset.shape

(50000, 4)

In [6]:
dataset['sentiment'].value_counts()

2    25374
1    24626
Name: sentiment, dtype: int64

In [7]:
dataset.loc[0,'Reviews']

'My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"'

In [8]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [9]:
dataset = dataset.iloc[:2500,:]
from tqdm import tqdm 
tqdm.pandas()
import re

In [10]:
dataset['Reviews'] = dataset['Reviews'].progress_apply(preprocess)

100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:04<00:00, 536.26it/s]


In [11]:
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
dataset_sent = dataset.loc[:,['sentiment','Reviews']]

In [13]:
dataset_sent.head()

,sentiment,Reviews
0,2,my lovely pat one great voices generation i l...
1,2,despite fact i played small portion game musi...
2,1,i bought charger jul 2003 worked ok the desig...
3,2,check maha energy s website their powerex mh-...
4,2,reviewed quite bit combo players hesitant due ...


In [14]:
!pip install textblob

You are using pip version 10.0.1, however version 20.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [15]:
import textblob

In [16]:
textblob.TextBlob("I hate this film totally not good").sentiment

Sentiment(polarity=-0.575, subjectivity=0.75)

In [17]:
textblob.TextBlob("I Love the film").sentiment.polarity

0.5

In [18]:
#vader 
#afinn
#Wornet

In [19]:
import pandas as pd

In [20]:
dataset = pd.read_csv(r"movie_reviews.csv.bz2")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [21]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [26]:
def convert(data):
    if data =="positive":
        return 1
    else: 
        return 0

In [27]:
dataset['sentiment']=dataset['sentiment'].apply(convert)

In [28]:
dataset['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [22]:
dataset['review'] = dataset['review'].progress_apply(preprocess)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [03:13<00:00, 258.07it/s]


In [29]:
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

In [32]:
train_Reviews = reviews[:35000]
train_sentiments = sentiments[:35000]

In [33]:
test_Reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tfidf = TfidfVectorizer()

In [35]:
cv = CountVectorizer(binary = False, min_df = 5, max_df = 0.5, ngram_range = (1,2))

In [36]:
cv_train_features = cv.fit_transform(train_Reviews)

In [37]:
cv_test_features = cv.transform(test_Reviews)

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
lr  = LogisticRegression(penalty = 'l2',max_iter= 500)

#Regularization
l1 = Lasso 
l2 = Ridge

In [42]:
lr.fit(cv_train_features,train_sentiments)

c:\users\aadhi\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [44]:
lr_predictions = lr.predict(cv_test_features)

In [45]:
labels = ['negative','positive']
print(classification_report(test_sentiments,lr_predictions))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90      7490
           1       0.90      0.91      0.90      7510

   micro avg       0.90      0.90      0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



In [46]:
pd.DataFrame(confusion_matrix(test_sentiments,lr_predictions),index = labels,columns = labels)

,negative,positive
negative,6735,755
positive,713,6797


In [47]:
#Assignment - Random Forest